In [1]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

# Import MNIST data
import tensorflow_datasets as tfds
mnist = tfds.load(name='mnist', split=['train', 'test'], data_dir="../data/")

# Define some parameters
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128

# Where to save TensorBoard model summaries
LOG_DIR = "../data/logs/RNN_with_summaries"

# Create placeholders for inputs, labels
_inputs = tf.compat.v1.placeholder(tf.float32,shape=[None, time_steps,
                                              element_size],
                                              name='inputs')
y = tf.compat.v1.placeholder(tf.float32, shape=[None, num_classes],
                                              name='labels')

In [2]:
batch_x, batch_y = mnist(batch_size)
# Reshape data to get 28 sequences of 28 pixels
batch_x = batch_x.reshape((batch_size, time_steps, element_size))

TypeError: 'list' object is not callable

In [3]:
# This helper function, taken from the official TensorFlow documentation,
# simply adds some ops that take care of logging summaries
def variable_summaries(var):
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.summary.scalar('mean', mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.summary.scalar('stddev', stddev)
      tf.summary.scalar('max', tf.reduce_max(var))
      tf.summary.scalar('min', tf.reduce_min(var))
      tf.summary.histogram('histogram', var)

In [4]:
# Weights and bias for input and hidden layer
with tf.name_scope('rnn_weights'):
    with tf.name_scope("W_x"):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
    with tf.name_scope("W_h"):
        Wh = tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        variable_summaries(Wh)
    with tf.name_scope("Bias"):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
def rnn_step(previous_hidden_state,x):

        current_hidden_state = tf.tanh(
            tf.matmul(previous_hidden_state, Wh) +
            tf.matmul(x, Wx) + b_rnn)

        return current_hidden_state

In [6]:
# Current input shape: (batch_size, time_steps, element_size)
processed_input = tf.transpose(_inputs, perm=[1, 0, 2])
# Current input shape now: (time_steps, batch_size, element_size)

initial_hidden = tf.zeros([batch_size,hidden_layer_size])
# Getting all state vectors across time
all_hidden_states = tf.scan(rnn_step,
                            processed_input,
                            initializer=initial_hidden,
                            name='states')

In [7]:
import numpy as np
import tensorflow as tf

elems = np.array(["T","e","n","s","o","r", " ", "F","l","o","w"])
scan_sum = tf.scan(lambda a, x: a + x, elems)

sess=tf.compat.v1.InteractiveSession()
sess.run(scan_sum)

array([b'T', b'Te', b'Ten', b'Tens', b'Tenso', b'Tensor', b'Tensor ',
       b'Tensor F', b'Tensor Fl', b'Tensor Flo', b'Tensor Flow'],
      dtype=object)

In [8]:
# Weights for output layers
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope("W_linear"):
        Wl = tf.Variable(tf.compat.v1.truncated_normal([hidden_layer_size,
                                              num_classes],
                                              mean=0,stddev=.01))
        variable_summaries(Wl)
    with tf.name_scope("Bias_linear"):
        bl = tf.Variable(tf.compat.v1.truncated_normal([num_classes],
                                             mean=0,stddev=.01))
        variable_summaries(bl)

# Apply linear layer to state vector
def get_linear_layer(hidden_state):

    return tf.matmul(hidden_state, Wl) + bl

with tf.name_scope('linear_layer_weights') as scope:
    # Iterate across time, apply linear layer to all RNN outputs
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    # Get last output
    output = all_outputs[-1]
    tf.summary.histogram('outputs', output)

In [9]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(
   tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))
    tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    # Using RMSPropOptimizer
    train_step = tf.compat.v1.train.RMSPropOptimizer(0.001, 0.9)\
                                   .minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(
                                tf.argmax(y,1), tf.argmax(output,1))

    accuracy = (tf.reduce_mean(
                       tf.cast(correct_prediction, tf.float32)))*100
    tf.summary.scalar('accuracy', accuracy)

# Merge all the summaries
merged = tf.compat.v1.summary.merge_all()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
# Get a small test set
test_data = mnist.reshape((-1, time_steps,
                                                     element_size))
test_label = mnist.labels[:batch_size]

with tf.compat.v1.Session() as sess:
    # Write summaries to LOG_DIR -- used by TensorBoard
    train_writer = tf.compat.v1.summary.FileWriter(LOG_DIR + '/train',
                                         graph=tf.compat.v1.get_default_graph())
    test_writer = tf.compat.v1.summary.FileWriter(LOG_DIR + '/test',
                                        graph=tf.compat.v1.get_default_graph())

    sess.run(tf.compat.v1.global_variables_initializer())

    for i in range(10000):

            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Reshape data to get 28 sequences of 28 pixels
            batch_x = batch_x.reshape((batch_size, time_steps,
                                       element_size))
            summary,_ = sess.run([merged,train_step],
                                feed_dict={_inputs:batch_x, y:batch_y})
            # Add to summaries
            train_writer.add_summary(summary, i)

            if i % 1000 == 0:
                acc,loss, = sess.run([accuracy,cross_entropy],
                                     feed_dict={_inputs: batch_x,
                                                y: batch_y})
                print ("Iter " + str(i) + ", Minibatch Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
            if i % 10:
                # Calculate accuracy for 128 MNIST test images and
                # add to summaries
                summary, acc = sess.run([merged, accuracy],
                                        feed_dict={_inputs: test_data,
                                                   y: test_label})
                test_writer.add_summary(summary, i)

    test_acc = sess.run(accuracy, feed_dict={_inputs: test_data,
                                             y: test_label})
    print ("Test Accuracy:", test_acc)

AttributeError: 'list' object has no attribute 'reshape'

In [12]:
import tensorflow as tf
import tensorflow_datasets as tfds
mnist = tfds.load(name='mnist', split=['train', 'test'], data_dir="../data/")

element_size = 28;time_steps = 28;num_classes = 10
batch_size = 128;hidden_layer_size = 128

_inputs = tf.compat.v1.placeholder(tf.float32,shape=[None, time_steps,
                                           element_size],
                                           name='inputs')
y = tf.compat.v1.placeholder(tf.float32, shape=[None, num_classes],name='inputs')

# TensorFlow built-in functions
rnn_cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(hidden_layer_size)
outputs, _ = tf.compat.v1.nn.dynamic_rnn(rnn_cell, _inputs, dtype=tf.float32)

Wl = tf.Variable(tf.compat.v1.truncated_normal([hidden_layer_size, num_classes],
                                     mean=0,stddev=.01))
bl = tf.Variable(tf.compat.v1.truncated_normal([num_classes],mean=0,stddev=.01))

def get_linear_layer(vector):
    return tf.matmul(vector, Wl) + bl

last_rnn_output = outputs[:,-1,:]
final_output = get_linear_layer(last_rnn_output)

softmax = tf.nn.softmax_cross_entropy_with_logits(logits=final_output,
                                                  labels=y)
cross_entropy = tf.reduce_mean(softmax)
train_step = tf.compat.v1.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(final_output,1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100

sess=tf.compat.v1.InteractiveSession()
sess.run(tf.compat.v1.global_variables_initializer())

test_data = mnist.test.images[:batch_size].reshape((-1,
                                            time_steps, element_size))
test_label = mnist.test.labels[:batch_size]

for i in range(3001):

       batch_x, batch_y = mnist.train.next_batch(batch_size)
       batch_x = batch_x.reshape((batch_size, time_steps, element_size))
       sess.run(train_step,feed_dict={_inputs:batch_x,
                                      y:batch_y})
       if i % 1000 == 0:
            acc = sess.run(accuracy, feed_dict={_inputs: batch_x,
                                                y: batch_y})
            loss = sess.run(cross_entropy,feed_dict={_inputs:batch_x,
                                                     y:batch_y})
            print ("Iter " + str(i) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

print ("Testing Accuracy:",
    sess.run(accuracy, feed_dict={_inputs: test_data, y: test_label}))

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
/home/af/Dokumenter/Programs/miniconda3/envs/LearningTensorFlow/lib/python3.8/site-packages/tensorflow/python/client/session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


AttributeError: 'list' object has no attribute 'test'

In [13]:
import numpy as np
import tensorflow as tf

batch_size = 128;embedding_dimension = 64;num_classes = 2
hidden_layer_size = 32;times_steps = 6;element_size = 1

In [15]:
digit_to_word_map = {1: "One", 2: "Two", 3: "Three", 4: "Four", 5: "Five", 6: "Six", 7: "Seven", 8: "Eight", 9: "Nine", 0: "PAD"}

even_sentences = []
odd_sentences = []
seqlens = []
for i in range(10000):
    rand_seq_len = np.random.choice(range(3,7))
    seqlens.append(rand_seq_len)
    rand_odd_ints = np.random.choice(range(1,10,2),
                                     rand_seq_len)
    rand_even_ints = np.random.choice(range(2,10,2),
                                      rand_seq_len)

    # Padding
    if rand_seq_len<6:
        rand_odd_ints = np.append(rand_odd_ints,
                                  [0]*(6-rand_seq_len))
        rand_even_ints = np.append(rand_even_ints,
                                   [0]*(6-rand_seq_len))

    even_sentences.append(" ".join([digit_to_word_map[r] for
                               r in rand_odd_ints]))
    odd_sentences.append(" ".join([digit_to_word_map[r] for
                              r in rand_even_ints]))

data = even_sentences+odd_sentences
# Same seq lengths for even, odd sentences
seqlens*=2

In [16]:
even_sentences[0:6]

['Seven Five Seven PAD PAD PAD',
 'Nine Seven Five One Three PAD',
 'Five One Nine PAD PAD PAD',
 'Five Nine One One One PAD',
 'One One Three One Five Seven',
 'Nine One Three Seven Nine Nine']

In [17]:
odd_sentences[0:6]

['Four Four Six PAD PAD PAD',
 'Six Four Four Six Four PAD',
 'Four Two Eight PAD PAD PAD',
 'Eight Eight Six Eight Eight PAD',
 'Two Eight Six Four Four Six',
 'Eight Two Four Two Four Two']

In [18]:
seqlens[0:6]

[3, 5, 3, 5, 6, 6]

In [19]:
# Map from words to indices
word2index_map ={}
index=0
for sent in data:
    for word in sent.lower().split():
        if word not in word2index_map:
            word2index_map[word] = index
            index+=1
# Inverse map
index2word_map = {index: word for word, index in word2index_map.items()}
vocabulary_size = len(index2word_map)

In [20]:
labels = [1]*10000 + [0]*10000
for i in range(len(labels)):
    label = labels[i]
    one_hot_encoding = [0]*2
    one_hot_encoding[label] = 1
    labels[i] = one_hot_encoding


data_indices = list(range(len(data)))
np.random.shuffle(data_indices)
data = np.array(data)[data_indices]

labels = np.array(labels)[data_indices]
seqlens = np.array(seqlens)[data_indices]
train_x = data[:10000]
train_y = labels[:10000]
train_seqlens = seqlens[:10000]

test_x = data[10000:]
test_y = labels[10000:]
test_seqlens = seqlens[10000:]

In [21]:
def get_sentence_batch(batch_size,data_x,
                       data_y,data_seqlens):
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    x = [[word2index_map[word] for word in data_x[i].lower().split()]
         for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x,y,seqlens

In [22]:
_inputs = tf.compat.v1.placeholder(tf.int32, shape=[batch_size,times_steps])
_labels = tf.compat.v1.placeholder(tf.float32, shape=[batch_size, num_classes])

# seqlens for dynamic calculation
_seqlens = tf.compat.v1.placeholder(tf.int32, shape=[batch_size])

In [23]:
with tf.name_scope("embeddings"):
    embeddings = tf.Variable(
        tf.compat.v1.random_uniform([vocabulary_size,
                           embedding_dimension],
                          -1.0, 1.0),name='embedding')
    embed = tf.nn.embedding_lookup(embeddings, _inputs)

In [24]:
with tf.compat.v1.variable_scope("lstm"):
    lstm_cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(hidden_layer_size,
                                             forget_bias=1.0)
    outputs, states = tf.compat.v1.nn.dynamic_rnn(lstm_cell, embed,
                                        sequence_length = _seqlens,
                                        dtype=tf.float32)

weights = {
    'linear_layer': tf.Variable(tf.compat.v1.truncated_normal([hidden_layer_size,
                                                     num_classes],
                                                     mean=0,stddev=.01))
}
biases = {
    'linear_layer':tf.Variable(tf.compat.v1.truncated_normal([num_classes],
                                                   mean=0,stddev=.01))
}

# Extract the last relevant output and use in a linear layer
final_output = tf.matmul(states[1],
                         weights["linear_layer"]) + biases["linear_layer"]
softmax = tf.nn.softmax_cross_entropy_with_logits(logits = final_output,
                                                  labels = _labels)
cross_entropy = tf.reduce_mean(softmax)

TypeError: ('Keyword argument not understood:', 'forget_bias')

In [25]:
train_step = tf.compat.v1.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(_labels,1),
                              tf.argmax(final_output,1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction,
                                   tf.float32)))*100

with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())

    for step in range(1000):
        x_batch, y_batch,seqlen_batch = get_sentence_batch(batch_size,
                                                           train_x,train_y,
                                                           train_seqlens)
        sess.run(train_step,feed_dict={_inputs:x_batch, _labels:y_batch,
                                       _seqlens:seqlen_batch})

        if step % 100 == 0:
            acc = sess.run(accuracy,feed_dict={_inputs:x_batch,
                                               _labels:y_batch,
                                               _seqlens:seqlen_batch})
            print("Accuracy at %d: %.5f" % (step, acc))

    for test_batch in range(5):
        x_test, y_test,seqlen_test = get_sentence_batch(batch_size,
                                                        test_x,test_y,
                                                        test_seqlens)
        batch_pred,batch_acc = sess.run([tf.argmax(final_output,1),
                                         accuracy],
                                        feed_dict={_inputs:x_test,
                                                   _labels:y_test,
                                                   _seqlens:seqlen_test})
        print("Test batch accuracy %d: %.5f" % (test_batch, batch_acc))

    output_example = sess.run([outputs],feed_dict={_inputs:x_test,
                                                   _labels:y_test,
                                                   _seqlens:seqlen_test})
    states_example = sess.run([states[1]],feed_dict={_inputs:x_test,
                                                   _labels:y_test,
                                                   _seqlens:seqlen_test})

InvalidArgumentError: You must feed a value for placeholder tensor 'inputs_1' with dtype float and shape [?,28,28]
	 [[node inputs_1 (defined at <ipython-input-12-494f03fd047f>:8) ]]

Original stack trace for 'inputs_1':
  File "/home/af/Dokumenter/Programs/miniconda3/envs/LearningTensorFlow/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/af/Dokumenter/Programs/miniconda3/envs/LearningTensorFlow/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/af/Dokumenter/Programs/miniconda3/envs/LearningTensorFlow/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/af/Dokumenter/Programs/miniconda3/envs/LearningTensorFlow/lib/python3.8/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/af/Dokumenter/Programs/miniconda3/envs/LearningTensorFlow/lib/python3.8/site-packages/i

In [26]:
seqlen_test[1]

NameError: name 'seqlen_test' is not defined

In [27]:
output_example[0][1][:6,0:3]

NameError: name 'output_example' is not defined

In [28]:
states_example[0][1][0:3]

NameError: name 'states_example' is not defined

In [29]:
num_LSTM_layers = 2
with tf.compat.v1.variable_scope("lstm"):

    lstm_cell_list = [tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_layer_size,forget_bias=1.0)
        for ii in range(num_LSTM_layers)]
    cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell(cells=lstm_cell_list,
        state_is_tuple=True)
    outputs, states = tf.compat.v1.nn.dynamic_rnn(cell, embed,
                                        sequence_length = _seqlens,
                                        dtype=tf.float32)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [30]:
# Extract the final state and use in a linear layer
final_output = tf.matmul(states[num_LSTM_layers-1][1],
                         weights["linear_layer"]) + biases["linear_layer"]

NameError: name 'weights' is not defined